In [1]:
import sys;
sys.path.insert(0, '..')

## Chapter 6 Code Snippets and Listings

### Finding good and bad outcomes with oracles (section 6.1.1)

Assume we want to find one special item in a list of $N = 8$ items, but we do not know where the item is. We can represent each of the items as an outcome of a quantum state with $n = 3$ qubits. Any randomly selected outcome has the same likelihood of being the good outcome. We represent this with equal amplitudes for each outcome:

In [2]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [3]:
from util import print_state_table

print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 



Let's look at an example of the classical implementation of a phase oracle from chapter 8. The function `predicate` identifies good outcome 3 as the only good outcome.

In [4]:
predicate = lambda k: True if k == 3 else False

We can use this predicate for the classical implementation of a phase oracle:

In [5]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1
            
oracle(state, predicate)

In [6]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 



### Inversion by the mean (section 6.1.2)

Listing 6.1 Classical implementation of the inversion operator

In [7]:
def inner(v1, v2):
    assert(len(v1) == len(v2))
    return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

Previous example three-qubit state obtained by applying an oracle that tags the outcome 3 to a state where the amplitudes are in equal superposition:

In [8]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy() # Copy the original state to use for the inversion

oracle(state, predicate)

Now, we can apply the inversion operator that reflects the current state into the original state (the state before the oracle was applied):

In [9]:
inversion(s, state) # The parameter s is the initial state before the oracle was applied

In [10]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
1        001     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
2        010     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
3        011     0.8839 + i0.0000       0.00°   0.8839     █████████████████████     0.7812
4        100     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
5        101     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
6        110     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
7        111     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312



Let's look at another example, with a random $n = 3$ qubit state and good outcome 5:

In [11]:
from util import generate_state

n = 3
state = generate_state(n)

In [12]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1850 - i0.1342    -144.40°   0.2285     █████                     0.0522
1        001     0.2070 + i0.0254       7.00°   0.2086     █████                     0.0435
2        010     0.2726 + i0.2780      45.56°   0.3894     █████████                 0.1516
3        011     0.4867 + i0.1080      12.51°   0.4985     ███████████               0.2485
4        100    -0.2028 + i0.0177     175.10°   0.2036     ████                      0.0414
5        101     0.3219 + i0.0871      15.14°   0.3335     ████████                  0.1112
6        110    -0.1712 - i0.3447    -116.41°   0.3849     █████████                 0.1481
7        111     0.3190 + i0.3188      44.98°   0.4509     ██████████                0.2034



In [13]:
s = state.copy()

predicate = lambda k: True if k == 5 else False
oracle(state, predicate)

In [14]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1850 - i0.1342    -144.40°   0.2285     █████                     0.0522
1        001     0.2070 + i0.0254       7.00°   0.2086     █████                     0.0435
2        010     0.2726 + i0.2780      45.56°   0.3894     █████████                 0.1516
3        011     0.4867 + i0.1080      12.51°   0.4985     ███████████               0.2485
4        100    -0.2028 + i0.0177     175.10°   0.2036     ████                      0.0414
5        101    -0.3219 - i0.0871    -164.86°   0.3335     ████████                  0.1112
6        110    -0.1712 - i0.3447    -116.41°   0.3849     █████████                 0.1481
7        111     0.3190 + i0.3188      44.98°   0.4509     ██████████                0.2034



Next, we perform the inversion:

In [15]:
inversion(s, state)

In [16]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1027 - i0.0745    -144.40°   0.1269     ███                       0.0161
1        001     0.1149 + i0.0141       7.00°   0.1158     ██                        0.0134
2        010     0.1513 + i0.1543      45.56°   0.2161     █████                     0.0467
3        011     0.2701 + i0.0599      12.50°   0.2767     ██████                    0.0766
4        100    -0.1126 + i0.0098     175.30°   0.113      ██                        0.0128
5        101     0.8226 + i0.2227      15.15°   0.8522     ████████████████████      0.7262
6        110    -0.0950 - i0.1913    -116.41°   0.2136     █████                     0.0456
7        111     0.1770 + i0.1769      44.98°   0.2503     ██████                    0.0626



**Visualizing inversion by the mean**

let's create a $n = 3$ qubit state in equal superposition and apply an oracle for good outcome 3:

In [17]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy()

predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

We can check that the mean of the amplitudes is equal to the quantity defined above for bad outcomes $k$:

In [18]:
from util import is_close

amplitude_mean = sum(state)/2**n

proj = inner(s, state)
for k in range(len(state)):
    if k != 3:
        assert is_close(proj*state[k], amplitude_mean)

Now, we can simulate the inversion by the mean with the following Python code:

In [19]:
for k in range(len(state)):
    state[k] = 2*amplitude_mean-state[k]

In [20]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
1        001     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
2        010     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
3        011     0.8839 + i0.0000       0.00°   0.8839     █████████████████████     0.7812
4        100     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
5        101     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
6        110     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
7        111     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312



### Putting it together: the Grover iterate (section 6.1.3)

Listing 6.2 Classical implementation of the Grover iterate

In [21]:
from math import cos

def grover_sim(state, predicate, iterations):
    s = state.copy()

    # Use the probability of measuring a good outcome to define an angle theta
    p = sum([abs(s[k])**2 for k in items])
    theta = asin(sqrt(p))
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        # The inner product after operator A is applied and the state after j Grover iterations is cos(2j theta)
        assert is_close(inner(s, state), cos(2*it*theta))

        p = sum([abs(state[k])**2 for k in items]) # Find the new probability of measuring a good outcome
        assert is_close(p, sin((2*it + 1)*theta)**2) # Check that the probability of good outcomes is sin^2((2j+1)theta)

In the case that the operator $A$ prepares a state with a uniform distribution, the magnitudes of good outcomes are given by the function below, where the parameter `n` is the number of qubits, `L` is the number of good outcomes, and `j` is the number of iterations.

In [22]:
from math import sin, asin

def target_amplitude_uniform(n, l, j):
    theta = asin(sqrt(l/2**n))
    return sin((2*j+1)*theta)/sqrt(l)

Let's apply one iteration of the Grover iterate to our example state, where $n = 3$ and the outcome 3 is the good outcome:

In [23]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 1)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 1))

In [24]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
1        001     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
2        010     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
3        011     0.8839 + i0.0000       0.00°   0.8839     █████████████████████     0.7812
4        100     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
5        101     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
6        110     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
7        111     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312



Let's apply another iteration:

In [25]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 2)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 2))

In [26]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
1        001    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
2        010    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
3        011     0.9723 + i0.0000       0.00°   0.9723     ███████████████████████   0.9453
4        100    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
5        101    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
6        110    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078
7        111    -0.0884 + i0.0000     180.00°   0.0884     ██                        0.0078



Let's try three iterations:

In [27]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 3)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 3))

In [28]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
1        001    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
2        010    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
3        011     0.5745 + i0.0000       0.00°   0.5745     █████████████             0.3301
4        100    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
5        101    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
6        110    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957
7        111    -0.3094 + i0.0000     180.00°   0.3094     ███████                   0.0957



Using the number of good outcomes, `L = len(items)`, and the total number of outcomes, `2**n`, we can find the optimal number of iterations with:

In [29]:
from math import floor, pi

num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

Let's look at another example, this time using a random two-qubit state and good outcome 1:

In [30]:
n = 3
items = [0, 1]
predicate = lambda i: True if i in items else False
iterations = int(floor(pi/4*sqrt(2**n/len(items))))

state = generate_state(n)
grover_sim(state, predicate, iterations)

In [31]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.4842 - i0.3511    -144.50°   0.5981     ██████████████            0.3577
1        001     0.5418 + i0.0665       7.00°   0.5459     █████████████             0.298 
2        010     0.1682 + i0.1715      45.56°   0.2403     █████                     0.0577
3        011     0.3003 + i0.0666      12.50°   0.3076     ███████                   0.0946
4        100    -0.1251 + i0.0109     175.20°   0.1256     ███                       0.0158
5        101     0.1986 + i0.0538      15.16°   0.2058     ████                      0.0424
6        110    -0.1056 - i0.2127    -116.40°   0.2375     █████                     0.0564
7        111     0.1968 + i0.1967      44.99°   0.2782     ██████                    0.0774



### A new implementation of the inversion operator (section 6.1.4)

In this section, we will use a randomly generated operator $A$. To create such an operator for $n$ qubits, we will use the function `random_transformation` defined in util.py. 

Let's define a random transformation (and its inverse) for $n = 3$ qubits:

In [32]:
from util import random_transformation

n = 3
f = random_transformation(n)
A = f[0]
A_inverse = f[1]

Let's look at the state prepared by this random operator:

In [33]:
from sim_core import init_state

state = init_state(n)
f[0](state)

In [34]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1307 + i0.2302     119.59°   0.2647     ██████                    0.0701
1        001    -0.3293 + i0.2192     146.35°   0.3956     █████████                 0.1565
2        010    -0.1697 + i0.0431     165.75°   0.1751     ████                      0.0307
3        011     0.2359 + i0.0759      17.84°   0.2478     █████                     0.0614
4        100    -0.0423 - i0.0510    -129.67°   0.0662     █                         0.0044
5        101    -0.2412 - i0.3226    -126.78°   0.4028     █████████                 0.1623
6        110     0.2784 + i0.1030      20.30°   0.2968     ███████                   0.0881
7        111     0.3755 - i0.5344     -54.91°   0.6531     ███████████████           0.4266



Listing 6.3 Function to perform the inversion operation using the operator $A$

In [35]:
from math import log2

def inversion_0_transformation(f, state):
    n = int(log2(len(state)))

    transform = f[0]
    inverse_transform = f[1]

    inverse_transform(state)
    inversion(init_state(n), state)
    transform(state)

Let's use the same operator $A$ we used to create the state above and apply an oracle for the good outcome 3:

In [36]:
predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

In [37]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1307 + i0.2302     119.59°   0.2647     ██████                    0.0701
1        001    -0.3293 + i0.2192     146.35°   0.3956     █████████                 0.1565
2        010    -0.1697 + i0.0431     165.75°   0.1751     ████                      0.0307
3        011    -0.2359 - i0.0759    -162.16°   0.2478     █████                     0.0614
4        100    -0.0423 - i0.0510    -129.67°   0.0662     █                         0.0044
5        101    -0.2412 - i0.3226    -126.78°   0.4028     █████████                 0.1623
6        110     0.2784 + i0.1030      20.30°   0.2968     ███████                   0.0881
7        111     0.3755 - i0.5344     -54.91°   0.6531     ███████████████           0.4266



Now, we can apply the inversion operation to the state:

In [38]:
inversion_0_transformation(f, state)

In [39]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.0986 + i0.1737     119.57°   0.1997     ████                      0.0399
1        001    -0.2484 + i0.1654     146.34°   0.2985     ███████                   0.0891
2        010    -0.1280 + i0.0325     165.75°   0.1321     ███                       0.0175
3        011     0.6497 + i0.2090      17.83°   0.6825     ████████████████          0.4658
4        100    -0.0319 - i0.0384    -129.72°   0.0499     █                         0.0025
5        101    -0.1820 - i0.2434    -126.79°   0.3039     ███████                   0.0923
6        110     0.2100 + i0.0777      20.30°   0.2239     █████                     0.0501
7        111     0.2833 - i0.4031     -54.90°   0.4927     ███████████               0.2428



### Quantum Oracle (section 6.2.1)

Listing 6.4 Function to create a phase oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [40]:
from sim_circuit import *

def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

### The inversion operator (section 6.2.2)

Listing 6.5 Function to create a circuit for `n` qubits that multiplies outcome 0 by -1

In [41]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

Listing 6.6 Function to create the inversion circuit

In [42]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

### Grover iterate (section 6.2.3)

Listing 6.7 Function to create the Grover iterate circuit for a given oracle `O` and operator `A`

In [43]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

### Putting it all together: Grover's Algorithm (section 6.2.4)

Listing 6.8 Function to create the magnitude amplification circuit

In [44]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    
    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

We can use the function `uniform` from chapter 4 to create a circuit $A$ that prepares a state with equal magnitudes:

In [45]:
def uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

With `uniform` as our operator $A$, $n = 3$ qubits, and good outcomes 1, 3, and 7, we can define the following circuit:

In [46]:
n = 3
items = [1, 3, 7]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

qc = grover_circuit(uniform(n), phase_oracle_match(n, items), num_iterations)

In [47]:
from util_qiskit import print_circuit

print_circuit(qc)

     ┌───┐                                         ┌───┐┌───┐       ┌───┐┌───┐
q_0: ┤ H ├──────■────────────────■───────────■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     ├───┤┌───┐ │     ┌───┐      │           │     ├───┤├───┤ │     ├───┤├───┤
q_1: ┤ H ├┤ X ├─■─────┤ X ├──────■───────────■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     ├───┤├───┤ │P(π) ├───┤┌───┐ │P(π) ┌───┐ │P(π) ├───┤├───┤ │P(π) ├───┤├───┤
q_2: ┤ H ├┤ X ├─■─────┤ X ├┤ X ├─■─────┤ X ├─■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     └───┘└───┘       └───┘└───┘       └───┘       └───┘└───┘       └───┘└───┘


We can check the amplitudes at each step using the reports generated with the following code:

In [48]:
for i in range(1, num_iterations + 1):
    for m in items:
        assert is_close(qc.reports[f'iteration_{i}'][2][m], (-1)**i * target_amplitude_uniform(n, len(items), i))

In [49]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
1        001    -0.5303 + i0.0000     180.00°   0.5303     ████████████              0.2812
2        010     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
3        011    -0.5303 + i0.0000     180.00°   0.5303     ████████████              0.2812
4        100     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
5        101     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
6        110     0.1768 + i0.0000       0.00°   0.1768     ████                      0.0312
7        111    -0.5303 + i0.0000     180.00°   0.5303     ████████████              0.2812

